# Selenium DOC dashboard scraper

THis notebook documents and executes the collection of daily COVID-19 data from the Pennsylvania DOC COVID-19 dashboard. Data is scraped from the DOC dashboard using Selenium and is then processed to conform to a standard dataframe format. 

Data fields collected in this notebook are only for incarcerated people (page 5 of the DOC dashboard), including positive tests, negative tests, pending tests, recovered cases and incarcerated person deaths. 


### Data collection procedure:
1. route selenium driver to DOC dashboard url
2. navigate driver to page 5 of dashboard
3. select date and SCI from dropdown menus
4. isolate and scrape relevant data fields
5. format and store relevant data fields

**NOTE** functions are defined out of order to allow for proper order of operations in loops

In [189]:
# import libraries

import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib import request
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from xml.etree.ElementTree import XML, fromstring

import time
import re

### 1. route selenium driver to DOC dashboard URL

In [328]:
# start up selenium
dash_url = "https://app.powerbigov.us/view?r=eyJrIjoiNWQ5YTQ4ZWUtY2NjMi00ZWRhLTgyNWQtYzAzNzc5NmYwMGIyIiwidCI6IjQxOGUyODQxLTAxMjgtNGRkNS05YjZjLTQ3ZmM1YTlhMWJkZSJ9"
driver = webdriver.Safari()
driver.get(dash_url)

# some div selectors

# change 12/21/20 - for sci_dropdown to "nth-child(49)" from "nth-child(46)" ??

sci_dropdown_scroll_area = 'document.querySelector("body > div:nth-child(49) > div.slicer-dropdown-content > div > div.slicerBody > div > div.scrollbar-inner.scroll-content.scroll-scrolly_visible")'
date_dropdown_scroll_area = 'document.querySelector("body > div:nth-child(47) > div.slicer-dropdown-content > div > div.slicerBody > div > div.scrollbar-inner.scroll-content.scroll-scrolly_visible")'

SessionNotCreatedException: Message: Could not create a session: The Safari instance is already paired with another WebDriver session.


In [289]:
# tracking data
all_data = []
bad_dates = []
date_list = []

### 2. navigate driver to page 5 of dashboard

In [278]:
def go_to_page_5():
    for i in range(4):
        driver.execute_script("document.querySelector('[title=\"Next Page\"]').click()")


### 4. Isolate and scrape content of svg data

In [279]:
def get_svg_content():
    result = None
    while result is None:
        try:
            d = driver.execute_script(' return document.querySelector("#pvExplorationHost > div > div > exploration > div > explore-canvas-modern > div > div.canvasFlexBox > div > div.displayArea.disableAnimations.fitToPage > div.visualContainerHost > visual-container-repeat > visual-container-modern:nth-child(6) > transform > div > div:nth-child(3) > div > visual-modern > div > svg")')
            content = [i.get_attribute('innerHTML') for i in d.find_elements_by_tag_name("*")]
            return content
        except:
             pass
    

### 3. select date and SCI from dropdown menus

In [194]:
# open dropdowns

def open_dropdowns():
    # open date dropdown
    driver.execute_script('document.querySelector("#pvExplorationHost > div > div > exploration > div > explore-canvas-modern > div > div.canvasFlexBox > div > div.displayArea.disableAnimations.fitToPage > div.visualContainerHost > visual-container-repeat > visual-container-modern:nth-child(7) > transform > div > div:nth-child(3) > div > visual-modern > div > div > div.slicer-content-wrapper > div > i").click()')
    # open sci dropdown
    driver.execute_script('document.querySelector("#pvExplorationHost > div > div > exploration > div > explore-canvas-modern > div > div.canvasFlexBox > div > div.displayArea.disableAnimations.fitToPage > div.visualContainerHost > visual-container-repeat > visual-container-modern:nth-child(5) > transform > div > div:nth-child(3) > div > visual-modern > div > div > div.slicer-content-wrapper > div").click()')

In [280]:
def roll_through_SCI(date):

    time.sleep(0.2)
    
    height = 200
    sci_list = []
    for d_ in range(4):
        
        # scroll to next portion
        driver.execute_script(f'{sci_dropdown_scroll_area}.scrollTop = {height*d_}')
        
        time.sleep(0.1)
        
        # parent node
        sci_scroll_area = driver.execute_script(f'return {sci_dropdown_scroll_area}')
        
        # sometimes it misses the selection, so try again
        try:
            current_sci = sci_scroll_area.find_elements_by_class_name('slicerText')
            current_sci[0].get_attribute('title')
        except:
            current_sci = sci_scroll_area.find_elements_by_class_name('slicerText')

        # for ever sci click it and run the svg collector
        for i in current_sci:
            sci = i.get_attribute('title')
            if sci not in sci_list:
                sci_list.append(sci)
                driver.execute_script(f"document.querySelector('[title=\"{sci}\"]').click()")
                
                # get data
                sci_dict = {"content":get_svg_content()}
                sci_dict['sci'] = sci
                sci_dict['date'] = date
                
                # save data
                all_data.append(sci_dict)


In [196]:

def roll_through_dates():
    height = 180
    for d in range(50):
        # 50 seems to capture everything
        
        # scroll to the next portion
        driver.execute_script(f'{date_dropdown_scroll_area}.scrollTop = {height*d}')
        
        # parent node
        date_scroll_area = driver.execute_script(f'return {date_dropdown_scroll_area}')

        # sometimes it misses the selection, so try again
        try:
            current_dates = date_scroll_area.find_elements_by_class_name('slicerText')
            current_dates[0].get_attribute('title')
        except:

            current_dates = date_scroll_area.find_elements_by_class_name('slicerText')
        
        # go through every date and click if it isn't already recorded
        for i in current_dates:
            date = i.get_attribute('title')
            if date not in date_list:
                date_list.append(date)
                driver.execute_script(f"document.querySelector('[title=\"{date}\"]').click()")

                # roll through all sci
                roll_through_SCI(date)
            if i == "(Blank)": # stop everything for messy code below
                return True


In [290]:
go_to_page_5() # go to page 5

In [291]:
open_dropdowns() # open the dropdown menus

### DANGER, this is bad code and is embarrassing. It was stopped and started multiple times to get all data. bad bad bad 

In [ ]:
done = None
cnt=0
while not done:
    try:
        val = roll_through_dates()
        cnt+=1
        if cnt>10:
            done = True
    except Exception as E:
        print(E)
        pass


In [292]:
date='12/21/2020'

In [293]:
driver.execute_script(f"document.querySelector('[title=\"{date}\"]').click()")

In [240]:
sci_list=['ALBION',
 'BENNER TOWNSHIP',
 'CAMBRIDGE SPRINGS',
 'CAMP HILL',
 'CENTRAL OFFICE',
 'CHESTER',
 'COAL TOWNSHIP',
 'DALLAS',
 'FAYETTE',
 'FOREST',
 'FRACKVILLE',
 'GREENE',
 'HOUTZDALE',
 'HUNTINGDON',
 'LAUREL HIGHLANDS',
 'MAHANOY',
 'MERCER',
 'MUNCY',
 'PHOENIX',
 'PINE GROVE',
 'QUEHANNA BOOTCAMP',
 'RETREAT',
 'ROCKVIEW',
 'SMITHFIELD',
 'SOMERSET',
 'WAYMART']

In [331]:
'''
for sci in sci_list:
    driver.execute_script(f"document.querySelector('[title=\"{sci}\"]').click()")
    # get data
    sci_dict = {"content":get_svg_content()}
    sci_dict['sci'] = sci
    sci_dict['date'] = date

    # save data
    all_data.append(sci_dict)
'''

'\nfor sci in sci_list:\n    driver.execute_script(f"document.querySelector(\'[title="{sci}"]\').click()")\n    # get data\n    sci_dict = {"content":get_svg_content()}\n    sci_dict[\'sci\'] = sci\n    sci_dict[\'date\'] = date\n\n    # save data\n    all_data.append(sci_dict)\n'

In [297]:
roll_through_SCI(date)

In [298]:
for sci in all_data:
    print(sci['sci'])
    elem=fromstring('<svg>{}</svg>'.format(sci['content'][1]))
    for p in elem.findall('g/path'):
        c, v = p.attrib['aria-label'].split()[:2]
        v=v.replace(',','')
        sci[c]=v
    print()

ALBION

BENNER TOWNSHIP

CAMBRIDGE SPRINGS

CAMP HILL

CENTRAL OFFICE

CHESTER

COAL TOWNSHIP

DALLAS

FAYETTE

FOREST

FRACKVILLE

GREENE

HOUTZDALE

HUNTINGDON

LAUREL HIGHLANDS

MAHANOY

MERCER

MUNCY

PHOENIX

PINE GROVE

QUEHANNA BOOTCAMP

RETREAT

ROCKVIEW

SMITHFIELD

SOMERSET

WAYMART



In [299]:
update_df=pd.DataFrame(all_data)

In [300]:
update_df=update_df.drop(columns='content')
update_df['sci']

0                ALBION
1       BENNER TOWNSHIP
2     CAMBRIDGE SPRINGS
3             CAMP HILL
4        CENTRAL OFFICE
5               CHESTER
6         COAL TOWNSHIP
7                DALLAS
8               FAYETTE
9                FOREST
10           FRACKVILLE
11               GREENE
12            HOUTZDALE
13           HUNTINGDON
14     LAUREL HIGHLANDS
15              MAHANOY
16               MERCER
17                MUNCY
18              PHOENIX
19           PINE GROVE
20    QUEHANNA BOOTCAMP
21              RETREAT
22             ROCKVIEW
23           SMITHFIELD
24             SOMERSET
25              WAYMART
Name: sci, dtype: object

In [301]:
update_df['sci']=update_df['sci'].str.title()

In [302]:
update_df

,sci,date,NEGATIVE,POSITIVE,PENDING,RECOVERED,DEATH
0,Albion,12/21/2020,960,318,234,150,NaN
1,Benner Township,12/21/2020,945,619,309,326,1
2,Cambridge Springs,12/21/2020,2079,1447,127,115,1
3,Camp Hill,12/21/2020,5105,787,4612,599,6
4,Central Office,12/21/2020,20,56,NaN,12,NaN
5,Chester,12/21/2020,884,730,508,684,4
6,Coal Township,12/21/2020,3251,500,3218,348,NaN
7,Dallas,12/21/2020,2030,2059,212,959,9
8,Fayette,12/21/2020,1079,315,179,160,1
9,Forest,12/21/2020,581,233,406,135,1


In [303]:
latest_df = pd.read_csv('https://raw.githubusercontent.com/jmparelman/PA-SCI_COVID19/main/data/latest_data/PA_DOC_testing_data.csv')

In [304]:
latest_df.columns

Index(['date', 'date.1', 'SCI', 'staff_positive', 'staff_negative',
       'staff_pending', 'staff_death', 'staff_recovered',
       'incarcerated_person_positive', 'incarcerated_person_negative',
       'incarcerated_person_pending', 'incarcerated_person_death',
       'incarcerated_person_recovered', 'test_transfer',
       'test_transfer_positive', 'test_release', 'test_release_positive',
       'test_hospital', 'test_hospital_positive', 'test_surveilance',
       'test_surveilance_positive', 'test_symptomatic',
       'test_symptomatic_positive', 'test_miscellaneous', 'date.1.1',
       'staff_positive_new', 'staff_negative_new', 'staff_pending_new',
       'staff_death_new', 'staff_recovered_new',
       'incarcerated_person_positive_new', 'incarcerated_person_negative_new',
       'incarcerated_person_pending_new', 'incarcerated_person_death_new',
       'incarcerated_person_recovered_new', 'test_transfer_new',
       'test_transfer_positive_new', 'test_release_new',
       'test

In [305]:
update_df.columns

Index(['sci', 'date', 'NEGATIVE', 'POSITIVE', 'PENDING', 'RECOVERED', 'DEATH'], dtype='object')

In [306]:
mapcols = {
    'sci': 'SCI', 
    
    'POSITIVE':'incarcerated_person_positive', 
    'NEGATIVE':'incarcerated_person_negative',
    'PENDING':'incarcerated_person_pending', 
    'DEATH':'incarcerated_person_death',
    'RECOVERED':'incarcerated_person_recovered'
    
}

In [307]:
update_df=update_df.rename(columns=mapcols)

In [308]:
def f(s):
    p=s.split('/') 
    return f"{p[2]}-{p[0]}-{p[1]}"

update_df['date']=update_df['date'].apply(f)

In [309]:
l2 = latest_df.copy()

In [310]:
for c in mapcols.values():
    if c!='SCI':
        update_df[c]=update_df[c].astype(float)

In [139]:
l2

,date,date.1,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,...,test_transfer_positive_new,test_release_new,test_release_positive_new,test_hospital_new,test_hospital_positive_new,test_surveilance_new,test_surveilance_positive_new,test_symptomatic_new,test_symptomatic_positive_new,test_miscellaneous_new
0,2020-04-07,2020-04-07,Rockview,NaN,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-07,2020-04-07,Albion,NaN,5.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-07,2020-04-07,Benner Township,NaN,4.0,1.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-07,2020-04-07,Cambridge Springs,NaN,NaN,2.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-07,2020-04-07,Camp Hill,2.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,2020-12-17,2020-12-17,Retreat,4.0,10.0,1.0,NaN,4.0,13.0,1048.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2779,2020-12-17,2020-12-17,Rockview,135.0,112.0,33.0,NaN,176.0,144.0,788.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2780,2020-12-17,2020-12-17,Smithfield,123.0,175.0,19.0,NaN,95.0,223.0,726.0,...,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,1.0,NaN
2781,2020-12-17,2020-12-17,Somerset,114.0,157.0,16.0,NaN,88.0,843.0,1418.0,...,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,1.0,NaN


In [140]:
new_df=pd.concat([l2,update_df])

In [141]:
new_df.columns

Index(['date', 'date.1', 'SCI', 'staff_positive', 'staff_negative',
       'staff_pending', 'staff_death', 'staff_recovered',
       'incarcerated_person_positive', 'incarcerated_person_negative',
       'incarcerated_person_pending', 'incarcerated_person_death',
       'incarcerated_person_recovered', 'test_transfer',
       'test_transfer_positive', 'test_release', 'test_release_positive',
       'test_hospital', 'test_hospital_positive', 'test_surveilance',
       'test_surveilance_positive', 'test_symptomatic',
       'test_symptomatic_positive', 'test_miscellaneous', 'date.1.1',
       'staff_positive_new', 'staff_negative_new', 'staff_pending_new',
       'staff_death_new', 'staff_recovered_new',
       'incarcerated_person_positive_new', 'incarcerated_person_negative_new',
       'incarcerated_person_pending_new', 'incarcerated_person_death_new',
       'incarcerated_person_recovered_new', 'test_transfer_new',
       'test_transfer_positive_new', 'test_release_new',
       'test

In [142]:
new_df

,date,date.1,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,...,test_transfer_positive_new,test_release_new,test_release_positive_new,test_hospital_new,test_hospital_positive_new,test_surveilance_new,test_surveilance_positive_new,test_symptomatic_new,test_symptomatic_positive_new,test_miscellaneous_new
0,2020-04-07,2020-04-07,Rockview,NaN,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-07,2020-04-07,Albion,NaN,5.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-07,2020-04-07,Benner Township,NaN,4.0,1.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-07,2020-04-07,Cambridge Springs,NaN,NaN,2.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-07,2020-04-07,Camp Hill,2.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,2020-12-18,NaN,Retreat,NaN,NaN,NaN,NaN,NaN,13.0,1048.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,2020-12-18,NaN,Rockview,NaN,NaN,NaN,NaN,NaN,144.0,788.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2020-12-18,NaN,Smithfield,NaN,NaN,NaN,NaN,NaN,222.0,728.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2020-12-18,NaN,Somerset,NaN,NaN,NaN,NaN,NaN,843.0,1418.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [311]:
def add_deltas(all_data):
    doc2_df = all_data.copy()
    doc2_df=doc2_df.drop(columns='date').reset_index()

    print(doc2_df.head())

    exclude_cols = ['SCI', 'date', 'date.1', 'date.1.1']

    cols_to_use = [c for c in doc2_df.columns if c not in exclude_cols]

    for col in cols_to_use:
       print('Calculating delta for', col)
       doc2_df[f'{col}_new'] = doc2_df.groupby('SCI')[col].diff()
    
    doc2_df=doc2_df.set_index(doc2_df['date'])

    return doc2_df

In [144]:
new_df=new_df.set_index(pd.DatetimeIndex(new_df['date']))

#new2_df = add_deltas(new_df)

In [145]:
new_df2 = add_deltas(new_df.drop(columns=['date.1']))

        date                SCI  staff_positive  staff_negative  \
0 2020-04-07           Rockview             NaN             1.0   
1 2020-04-07             Albion             NaN             5.0   
2 2020-04-07    Benner Township             NaN             4.0   
3 2020-04-07  Cambridge Springs             NaN             NaN   
4 2020-04-07          Camp Hill             2.0             1.0   

   staff_pending  staff_death  staff_recovered  incarcerated_person_positive  \
0            2.0          NaN              NaN                           NaN   
1            2.0          NaN              NaN                           NaN   
2            1.0          NaN              NaN                           NaN   
3            2.0          NaN              NaN                           NaN   
4            NaN          NaN              NaN                           NaN   

   incarcerated_person_negative  incarcerated_person_pending  ...  \
0                           NaN                

In [312]:
agg=latest_df.copy()

In [313]:
agg

,date,date.1,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,...,test_transfer_positive_new,test_release_new,test_release_positive_new,test_hospital_new,test_hospital_positive_new,test_surveilance_new,test_surveilance_positive_new,test_symptomatic_new,test_symptomatic_positive_new,test_miscellaneous_new
0,2020-04-07,2020-04-07,Rockview,NaN,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-07,2020-04-07,Albion,NaN,5.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-07,2020-04-07,Benner Township,NaN,4.0,1.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-07,2020-04-07,Cambridge Springs,NaN,NaN,2.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-07,2020-04-07,Camp Hill,2.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,2020-12-18,2020-12-18,Retreat,4.0,10.0,1.0,NaN,4.0,13.0,1048.0,...,0.0,-7.0,0.0,0.0,0.0,156.0,0.0,0.0,0.0,NaN
2804,2020-12-18,2020-12-18,Rockview,209.0,148.0,32.0,NaN,177.0,144.0,788.0,...,-1.0,-75.0,-4.0,-11.0,-1.0,231.0,26.0,-17.0,-11.0,NaN
2805,2020-12-18,2020-12-18,Smithfield,186.0,211.0,19.0,NaN,97.0,222.0,728.0,...,-5.0,-63.0,-2.0,-4.0,-2.0,344.0,45.0,-14.0,-5.0,NaN
2806,2020-12-18,2020-12-18,Somerset,164.0,192.0,15.0,NaN,94.0,843.0,1418.0,...,0.0,-90.0,-4.0,-30.0,-1.0,361.0,93.0,-36.0,-15.0,NaN


In [314]:
cols_to_drop = [c for c in agg.columns if c.endswith('_new') or c.endswith('_D') or c.startswith('date.1')]

agg = agg.drop(columns=cols_to_drop)

new_df = pd.concat([agg,update_df])

new_df=new_df.set_index(pd.DatetimeIndex(new_df['date']))

new2_df = add_deltas(new_df)


        date                SCI  staff_positive  staff_negative  \
0 2020-04-07           Rockview             NaN             1.0   
1 2020-04-07             Albion             NaN             5.0   
2 2020-04-07    Benner Township             NaN             4.0   
3 2020-04-07  Cambridge Springs             NaN             NaN   
4 2020-04-07          Camp Hill             2.0             1.0   

   staff_pending  staff_death  staff_recovered  incarcerated_person_positive  \
0            2.0          NaN              NaN                           NaN   
1            2.0          NaN              NaN                           NaN   
2            1.0          NaN              NaN                           NaN   
3            2.0          NaN              NaN                           NaN   
4            NaN          NaN              NaN                           NaN   

   incarcerated_person_negative  incarcerated_person_pending  ...  \
0                           NaN                

In [315]:
new2_df.columns

Index(['date', 'SCI', 'staff_positive', 'staff_negative', 'staff_pending',
       'staff_death', 'staff_recovered', 'incarcerated_person_positive',
       'incarcerated_person_negative', 'incarcerated_person_pending',
       'incarcerated_person_death', 'incarcerated_person_recovered',
       'test_transfer', 'test_transfer_positive', 'test_release',
       'test_release_positive', 'test_hospital', 'test_hospital_positive',
       'test_surveilance', 'test_surveilance_positive', 'test_symptomatic',
       'test_symptomatic_positive', 'test_miscellaneous', 'staff_positive_new',
       'staff_negative_new', 'staff_pending_new', 'staff_death_new',
       'staff_recovered_new', 'incarcerated_person_positive_new',
       'incarcerated_person_negative_new', 'incarcerated_person_pending_new',
       'incarcerated_person_death_new', 'incarcerated_person_recovered_new',
       'test_transfer_new', 'test_transfer_positive_new', 'test_release_new',
       'test_release_positive_new', 'test_hospit

In [316]:
new2_df.shape

(2834, 44)

In [317]:
latest_df.shape

(2808, 46)

In [318]:
new2_df.head()

,date,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,incarcerated_person_pending,...,test_transfer_positive_new,test_release_new,test_release_positive_new,test_hospital_new,test_hospital_positive_new,test_surveilance_new,test_surveilance_positive_new,test_symptomatic_new,test_symptomatic_positive_new,test_miscellaneous_new
date,,,,,,,,,,,,,,,,,,,,,
2020-04-07,2020-04-07,Rockview,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,2020-04-07,Albion,NaN,5.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,2020-04-07,Benner Township,NaN,4.0,1.0,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,2020-04-07,Cambridge Springs,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,2020-04-07,Camp Hill,2.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
new2_df.tail()

,date,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,incarcerated_person_pending,...,test_transfer_positive_new,test_release_new,test_release_positive_new,test_hospital_new,test_hospital_positive_new,test_surveilance_new,test_surveilance_positive_new,test_symptomatic_new,test_symptomatic_positive_new,test_miscellaneous_new
date,,,,,,,,,,,,,,,,,,,,,
2020-12-21,2020-12-21,Retreat,NaN,NaN,NaN,NaN,NaN,29.0,1088.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-21,2020-12-21,Rockview,NaN,NaN,NaN,NaN,NaN,980.0,1380.0,292.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-21,2020-12-21,Smithfield,NaN,NaN,NaN,NaN,NaN,966.0,1572.0,771.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-21,2020-12-21,Somerset,NaN,NaN,NaN,NaN,NaN,1499.0,2186.0,1016.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-21,2020-12-21,Waymart,NaN,NaN,NaN,NaN,NaN,1213.0,4008.0,215.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [332]:
new2_df.to_csv('/Users/mattodonnell/Downloads/PA_DOC_testing_data_SCRAPED.csv')

In [321]:
new2_df.columns

Index(['date', 'SCI', 'staff_positive', 'staff_negative', 'staff_pending',
       'staff_death', 'staff_recovered', 'incarcerated_person_positive',
       'incarcerated_person_negative', 'incarcerated_person_pending',
       'incarcerated_person_death', 'incarcerated_person_recovered',
       'test_transfer', 'test_transfer_positive', 'test_release',
       'test_release_positive', 'test_hospital', 'test_hospital_positive',
       'test_surveilance', 'test_surveilance_positive', 'test_symptomatic',
       'test_symptomatic_positive', 'test_miscellaneous', 'staff_positive_new',
       'staff_negative_new', 'staff_pending_new', 'staff_death_new',
       'staff_recovered_new', 'incarcerated_person_positive_new',
       'incarcerated_person_negative_new', 'incarcerated_person_pending_new',
       'incarcerated_person_death_new', 'incarcerated_person_recovered_new',
       'test_transfer_new', 'test_transfer_positive_new', 'test_release_new',
       'test_release_positive_new', 'test_hospit

In [326]:
new2_df.loc['2020-12-21'][['SCI','incarcerated_person_positive','incarcerated_person_positive_new']]

,SCI,incarcerated_person_positive,incarcerated_person_positive_new
date,,,
2020-12-21,Albion,318.0,308.0
2020-12-21,Benner Township,619.0,460.0
2020-12-21,Cambridge Springs,1447.0,260.0
2020-12-21,Camp Hill,787.0,612.0
2020-12-21,Central Office,56.0,NaN
2020-12-21,Chester,730.0,552.0
2020-12-21,Coal Township,500.0,304.0
2020-12-21,Dallas,2059.0,983.0
2020-12-21,Fayette,315.0,300.0
